This notebook tests if a cell type has a response to stimulus

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.io

from janelia_core.stats.permutation_tests import paired_grouped_perm_test
from keller_zlatic_vnc.data_processing import count_unique_subjs_per_transition
from keller_zlatic_vnc.data_processing import extract_transitions
from keller_zlatic_vnc.data_processing import generate_transition_dff_table
from keller_zlatic_vnc.data_processing import read_raw_transitions_from_excel
from keller_zlatic_vnc.data_processing import recode_beh

/Users/bishopw/Documents/Janelia_Research/Projects/janelia_core/janelia_core/fileio/exp_reader.py:21: UserWarning: Unable to locate pyklb module.  Will not be able to read in .klb files.
  warnings.warn('Unable to locate pyklb module.  Will not be able to read in .klb files.')


## Options for analysis

In [3]:
# Here we specify the location of the data

data_folder = r'/Users/bishopw/Documents/Janelia_Research/Projects/keller_zlatic_vnc/data/extracted_dff_v2'
transition_file = 'transition_list.xlsx'

a00c_a4_act_data_file = 'A00c_activity_A4.mat'
a00c_a9_act_data_file = 'A00c_activity_A9.mat'

basin_a4_act_data_file = 'Basin_activity_A4.mat'
basin_a9_act_data_file = 'Basin_activity_A9.mat'

handle_a4_act_data_file = 'Handle_activity_A4.mat'
handle_a9_act_data_file = 'Handle_activity_A9.mat'

# =====================================================================
# Here, we specify how we want to filter the data when fitting models.  

# Cell types are tuples of form (cell type, list of cell ids).  In place of a list of cell ids, the string 'all'
# indicates we are using all cell ids
cell_type = ('handle', 'all')

manip_type = 'A4+A9'#, 'A9', 'A4+A9']
cut_off_time = 3.656

# ======================================================================================================================
# Here we specify the remaining parameters, common to all analyses

# Alpha value for forming confidence intervals and testing for significance
alpha = .05

## Prepare data

In [4]:
raw_trans = read_raw_transitions_from_excel(pathlib.Path(data_folder) / transition_file)

# Recode behavioral annotations
raw_trans = recode_beh(raw_trans, 'Beh Before')
raw_trans = recode_beh(raw_trans, 'Beh After')

# Read in neural activity
if cell_type[0] == 'a00c':
    a4_act_file = a00c_a4_act_data_file
    a9_act_file = a00c_a9_act_data_file
elif cell_type[0] == 'basin':
    a4_act_file = basin_a4_act_data_file
    a9_act_file = basin_a9_act_data_file
elif cell_type[0] == 'handle':
    a4_act_file = handle_a4_act_data_file
    a9_act_file = handle_a9_act_data_file
else:
    raise (ValueError('The cell type ' + cell_type + ' is not recogonized.'))

a4_act = scipy.io.loadmat(pathlib.Path(data_folder) / a4_act_file, squeeze_me=True)
a9_act = scipy.io.loadmat(pathlib.Path(data_folder) / a9_act_file, squeeze_me=True)

# Correct mistake in labeling if we need to
if cell_type[0] == 'basin' or cell_type[0] == 'handle':
    ind = np.argwhere(a4_act['newTransitions'] == '0824L2CL')[1][0]
    a4_act['newTransitions'][ind] = '0824L2-2CL'
    
# Extract transitions
trans = extract_transitions(raw_trans, cut_off_time)

# Generate table of data
a4table = generate_transition_dff_table(act_data=a4_act, trans=trans)
a9table = generate_transition_dff_table(act_data=a9_act, trans=trans)

# Put the tables together
a4table['man_tgt'] = 'A4'
a9table['man_tgt'] = 'A9'
data = a4table.append(a9table, ignore_index=True)

## Down select for manipulation target if needed

In [5]:
if manip_type == 'A4':
    data = data[data['man_tgt'] == 'A4']
elif manip_type == 'A9':
    data = data[data['man_tgt'] == 'A9']

## Down select for cell id

In [6]:
cell_ids = cell_type[1]
if isinstance(cell_ids, list):
    keep_rows = data['cell_id'].apply(lambda x: x in set(cell_ids))
    data = data[keep_rows]
    print('Using only cell ids ' + str(cell_ids) + ', leaving ' + str(len(data)) + ' data rows.')
else:
    print('Using all cell ids, leaving ' + str(len(data)) + ' data rows.')

Using all cell ids, leaving 3213 data rows.


## Pull out $\Delta F/F$ before and after manipulation

In [7]:
dff_after = data['dff_after'].to_numpy()
dff_before = data['dff_before'].to_numpy()


## Find grouping of data by subject

In [8]:
# Find grouping of data by subject
unique_ids = data['subject_id'].unique()
g = np.zeros(len(data))
for u_i, u_id in enumerate(unique_ids):
    g[data['subject_id'] == u_id] = u_i

## Calculate statistics

In [9]:
t, p = paired_grouped_perm_test(x0 = dff_before, x1 = dff_after, grp_ids=g, n_perms=1000)

In [10]:
print('Diff in Delta F/F: ' + str(t) + ', p=' + str(p))

Diff in Delta F/F: 0.457603115626508, p=0.0
